In [27]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy
from torchsummary import summary

In [28]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import random

In [29]:
import deeplake
ds = deeplake.load('hub://activeloop/wiki-art')

|

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/wiki-art



\

hub://activeloop/wiki-art loaded successfully.



In [30]:
tform = transforms.Compose([
    transforms.ToPILImage(), # Must convert to PIL image for subsequent operations to run
    transforms.ToTensor(),
    transforms.Resize((600,600)),
])

dataloader= ds.pytorch(batch_size=16, num_workers=2, 
    transform = {'images': tform, 'labels': None}, shuffle=True)

In [31]:
len_data = len(dataloader.dataset)
len_data

81433

In [32]:
for data in dataloader:
    print(data)    
    
    break



Please wait, filling up the shuffle buffer with samples.:   0%| | 0.00/1.91G [00Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x10f4e4d60><function _MultiProcessingDataLoaderIter.__del__ at 0x10f4e4d60>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/sarthakkapila/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
  File "/Users/sarthakkapila/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
        self._shutdown_workers()self._shutdown_workers()

  File "/Users/sarthakkapila/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
  File "/Users/sarthakkapila/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
        if w.is_alive():if w.is_alive():

              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/sarthak

Shuffle buffer filling is complete.
IterableOrderedDict([('images', tensor([[[[0.2752, 0.2854, 0.2994,  ..., 0.2093, 0.1799, 0.1885],
          [0.2277, 0.2334, 0.2538,  ..., 0.2075, 0.1690, 0.1703],
          [0.1870, 0.1861, 0.2119,  ..., 0.1941, 0.1608, 0.1640],
          ...,
          [0.4216, 0.4610, 0.4088,  ..., 0.5116, 0.5233, 0.5467],
          [0.4376, 0.4484, 0.4273,  ..., 0.5310, 0.5070, 0.4978],
          [0.4824, 0.4696, 0.4828,  ..., 0.5547, 0.5045, 0.4660]],

         [[0.2046, 0.2148, 0.2291,  ..., 0.1623, 0.1341, 0.1484],
          [0.1571, 0.1628, 0.1835,  ..., 0.1604, 0.1232, 0.1302],
          [0.1164, 0.1155, 0.1416,  ..., 0.1470, 0.1150, 0.1239],
          ...,
          [0.2708, 0.2957, 0.2243,  ..., 0.3632, 0.3730, 0.3934],
          [0.2760, 0.2832, 0.2492,  ..., 0.3792, 0.3526, 0.3377],
          [0.3115, 0.2931, 0.3051,  ..., 0.4002, 0.3441, 0.3016]],

         [[0.1889, 0.1996, 0.2196,  ..., 0.1765, 0.1418, 0.1491],
          [0.1415, 0.1475, 0.1740,  ...,

In [21]:
channels = 3
kernel1 = 5
kernel2 = 5 

kernel1_n = 16
kernel2_n = 32

stride = 1
padding = 0

kernel_pool = 2
n_linear = 64                              
input_height = 600
input_width = 600

conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=channels, out_channels=kernel1_n, kernel_size=kernel1, stride=stride, padding=padding, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=kernel_pool, stride=None, padding=0, dilation=1),
    
#             nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5,5), stride=1, padding=0, bias=True),
#             nn.Sigmoid(),
#             nn.MaxPool2d(kernel_size=kernel_pool, stride=None, padding=0, dilation=1),
)

conv_output_height = ((input_height - kernel1 + 2*padding) // stride + 1) // kernel_pool
conv_output_width = ((input_width - kernel1 + 2*padding) // stride + 1) // kernel_pool

linear_layer = nn.Linear(in_features= kernel1_n * conv_output_height * conv_output_width, out_features=n_linear)


model = nn.Sequential(
    conv_layer,
    nn.Flatten(),
    linear_layer
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001 (for now)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


summary(model, input_size=(3, 600 , 600)), model


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 596, 596]           1,216
              ReLU-2         [-1, 16, 596, 596]               0
         MaxPool2d-3         [-1, 16, 298, 298]               0
           Flatten-4              [-1, 1420864]               0
            Linear-5                   [-1, 64]      90,935,360
Total params: 90,936,576
Trainable params: 90,936,576
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 4.12
Forward/backward pass size (MB): 108.40
Params size (MB): 346.90
Estimated Total Size (MB): 459.42
----------------------------------------------------------------


(None,
 Sequential(
   (0): Sequential(
     (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
     (1): ReLU()
     (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
   (1): Flatten(start_dim=1, end_dim=-1)
   (2): Linear(in_features=1420864, out_features=64, bias=True)
 ))

In [26]:
max_steps = 10
losses = []

for epoch in range(max_steps):  
    for i, (images, labels) in enumerate(dataloader):
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item()) 
        
        if i % 100 == 0: 
            print(f'Epoch [{epoch+1}/{max_steps}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')


Please wait, filling up the shuffle buffer with samples.:   0%| | 0.00/1.91G [00ERROR: Unexpected segmentation fault encountered in worker.
 ERROR: Unexpected segmentation fault encountered in worker.
 

RuntimeError: DataLoader worker (pid(s) 44328, 44329) exited unexpectedly